In [3]:
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load the BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr = 2e-5)

# Load the dataset and tokenizer
from transformers import TextClassificationDataset
dataset = TextClassificationDataset(
    tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased"),
    data_dir = "path/to/dataset",
    max_len = 512,
)

# Set up the data loader
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size = 32, shuffle = True)

# Fine-tune the model
for epoch in range(3):
    for i, (input_ids, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = model(input_ids, labels = labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1} Loss {loss.item()}')
